# 라이브러리 로딩

In [1]:
import numpy as np # Numpy
import pandas as pd # Pandas
import matplotlib as mpl #Matplotlib 세팅용
import matplotlib.pyplot as plt # 시각화 도구
import seaborn as sns # 시각화 도구
from sklearn.model_selection import train_test_split # 데이터셋 분리
from sklearn.model_selection import KFold # KFold 교차검증
from sklearn.cluster import KMeans # 클러스터링
from sklearn.metrics import silhouette_score # 실루엣 점수
import xgboost as xgb # XGBoost
from sklearn.model_selection import GridSearchCV # 그리드 서치
from sklearn.metrics import accuracy_score, precision_score # 평가 지표
from sklearn.metrics import recall_score, confusion_matrix, roc_auc_score, f1_score # 평가 지표
from imblearn.combine import SMOTEENN, SMOTETomek # 복합샘플링
from hyperopt import hp, fmin, tpe, Trials # HyperOPT

import warnings # 경고문 제거용


%matplotlib inline
%config Inlinebackend.figure_format = 'retina'

# 한글 폰트 설정
mpl.rc('font', family='D2Coding')
# 유니코드에서 음수 부호 설정
mpl.rc('axes', unicode_minus = False)

warnings.filterwarnings('ignore')
sns.set(font="D2Coding", rc={"axes.unicode_minus":False}, style='darkgrid')
plt.rc('figure', figsize=(10,8))

# 데이터 불러오기

In [2]:
data = pd.read_excel('train_test_na_filled.xlsx', sheet_name='Train')

# 전처리

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 18 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8693 non-null   object 
 2   CryoSleep     8693 non-null   bool   
 3   Cabin1        8590 non-null   object 
 4   Cabin2        8590 non-null   float64
 5   Combi         8590 non-null   object 
 6   Cabin3        8590 non-null   object 
 7   Cabin         8590 non-null   object 
 8   Destination   8693 non-null   object 
 9   Age           8693 non-null   int64  
 10  VIP           8693 non-null   bool   
 11  RoomService   8693 non-null   int64  
 12  FoodCourt     8693 non-null   int64  
 13  ShoppingMall  8693 non-null   int64  
 14  Spa           8693 non-null   int64  
 15  VRDeck        8693 non-null   int64  
 16  Name          8493 non-null   object 
 17  Transported   8693 non-null   bool   
dtypes: bool(3), float64(1), int6

## 필요없는 features 제거

In [4]:
# 필요없는 features 제거
data.drop(['PassengerId', 'Cabin', 'Cabin1', 'Cabin2', 'Name',], axis=1, inplace=True)

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   HomePlanet    8693 non-null   object
 1   CryoSleep     8693 non-null   bool  
 2   Combi         8590 non-null   object
 3   Cabin3        8590 non-null   object
 4   Destination   8693 non-null   object
 5   Age           8693 non-null   int64 
 6   VIP           8693 non-null   bool  
 7   RoomService   8693 non-null   int64 
 8   FoodCourt     8693 non-null   int64 
 9   ShoppingMall  8693 non-null   int64 
 10  Spa           8693 non-null   int64 
 11  VRDeck        8693 non-null   int64 
 12  Transported   8693 non-null   bool  
dtypes: bool(3), int64(6), object(4)
memory usage: 704.7+ KB


## 처리하기 힘든 결측값 제거

In [6]:
data.isna().sum()

HomePlanet        0
CryoSleep         0
Combi           103
Cabin3          103
Destination       0
Age               0
VIP               0
RoomService       0
FoodCourt         0
ShoppingMall      0
Spa               0
VRDeck            0
Transported       0
dtype: int64

In [7]:
# 결측값들 제거(Cabin)
data.dropna(axis=0, inplace=True)

## Boolean 캐스팅

In [8]:
# Cabin3의 값을 변환
data['Cabin3'].replace({'P': True,'S': False}, inplace=True)
data['Cabin3'] = data['Cabin3'].astype(bool)

## 원핫인코딩

In [9]:
# 원핫인코딩
train_encoding = pd.get_dummies(data['HomePlanet'])
data=data.drop('HomePlanet',axis=1)
data = data.join(train_encoding)

train_encoding = pd.get_dummies(data['Destination'])
data=data.drop('Destination',axis=1)
data = data.join(train_encoding)

train_encoding = pd.get_dummies(data['Combi'])
data=data.drop('Combi',axis=1)
data = data.join(train_encoding)

In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8590 entries, 0 to 8692
Columns: 4469 entries, CryoSleep to T3
dtypes: bool(4), int64(6), uint8(4459)
memory usage: 37.3 MB


## 스케일링

In [11]:
# 스케일링(Cabin2)
col = ['Age', 'RoomService','FoodCourt','ShoppingMall','Spa','VRDeck']
def data_scaled(df, col):
    for i in col:
        data_mean = df[i].mean()
        data_std = df[i].std()
        scaled = (df[i]-data_mean)/data_std
        df[i]=scaled
    return df

In [12]:
data_scaled(data, col)

,CryoSleep,Cabin3,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,...,G993,G994,G995,G996,G998,G999,T0,T1,T2,T3
0,False,True,0.712274,False,-0.333743,-0.280785,-0.282832,-0.271469,-0.263361,False,...,0,0,0,0,0,0,0,0,0,0
1,False,False,-0.332624,False,-0.168530,-0.275148,-0.241196,0.220460,-0.224520,True,...,0,0,0,0,0,0,0,0,0,0
2,False,False,2.035811,True,-0.268567,1.959032,-0.282832,5.745469,-0.220106,False,...,0,0,0,0,0,0,0,0,0,0
3,False,False,0.294315,False,-0.333743,0.522818,0.335048,2.711463,-0.092988,False,...,0,0,0,0,0,0,0,0,0,0
4,False,False,-0.889902,False,0.125518,-0.236941,-0.031350,0.234796,-0.261596,True,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,False,True,0.851594,True,-0.333743,3.990274,-0.282832,1.200732,-0.198037,False,...,0,0,0,0,0,0,0,0,0,0
8689,True,False,-0.750583,False,-0.333743,-0.280785,-0.282832,-0.271469,-0.263361,False,...,0,0,0,0,0,0,0,0,0,0
8690,False,False,-0.193304,False,-0.333743,-0.280785,2.834877,-0.270573,-0.263361,True,...,0,0,0,0,0,0,0,0,0,0
8691,False,False,0.224655,False,-0.333743,0.376253,-0.282832,0.044835,2.592370,False,...,0,0,0,0,0,0,0,0,0,0


In [28]:
data.columns

Index(['CryoSleep', 'Cabin3', 'Age', 'VIP', 'RoomService', 'FoodCourt',
       'ShoppingMall', 'Spa', 'VRDeck', 'Transported',
       ...
       'G993', 'G994', 'G995', 'G996', 'G998', 'G999', 'T0', 'T1', 'T2', 'T3'],
      dtype='object', length=4469)

# 데이터셋 분리

In [13]:
X_train, X_test, y_train, y_test = train_test_split(data.drop(['Transported'], axis=1),
                                                    data.Transported, random_state=109)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train)

# XGBoost

In [14]:
xgb_search_space = {'max_depth': hp.quniform('max_depth', 5, 15, 1),
                   'min_child_weight': hp.quniform('min_child_weight', 1, 20, 1),
                   'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 0.95),
                   'learning_rate': hp.uniform('learning_rate', 0.01, 0.4),
                   'gamma': hp.uniform('gamma', 0, 4)}

In [15]:
# fmin()에서 호출 시 search_space 값으로 XGBClassifier 교차 검증 학습 후 -1 * roc_auc 평균 값을 반환
def bin_objective_func(search_space):
    xgb_clf = xgb.XGBClassifier(n_estimators=100, max_depth=int(search_space['max_depth']),
                            min_child_weight=int(search_space['min_child_weight']),
                            colsample_bytree=search_space['colsample_bytree'],
                            learning_rate=search_space['learning_rate'],
                            gamma=search_space['gamma'])
    
    # 3개 k-fold 방식으로 평가된 roc_auc 지표를 담는 list
    roc_auc_list = []
    
    # 3개 k-fold 방식 적용
    kf = KFold(n_splits=3)
    
    # X_train을 다시 학습과 검증용 데이터로 분리
    for tr_index, val_index in kf.split(X_train):
        # kf.split(X_train)으로 추출된 학습과 검증 index 값으로 학습과 검증 데이터 세트 분리
        X_tr, y_tr = X_train.iloc[tr_index], y_train.iloc[tr_index]
        X_val, y_val = X_train.iloc[val_index], y_train.iloc[val_index]
        
        # early stopping은 30회로 설정하고 추출된 학습과 검증 데이터로 XGBClassifier 학습 수행
        xgb_clf.fit(X_tr, y_tr, early_stopping_rounds=30, eval_metric="auc",
                   eval_set=[(X_tr, y_tr), (X_val, y_val)])
        
        # 1로 예측한 확률값 추출 후 roc auc 계산하고 평균 roc auc 계산을 위해 list에 결과값 담음.
        score = roc_auc_score(y_val, xgb_clf.predict_proba(X_val)[:,1])
        roc_auc_list.append(score)
        
    # 3개 k-fold로 계산된 roc_auc 값의 평균값을 반환하되,
    # HyperOPT는 목적함수의 최솟값을 위한 입력값을 찾으므로 -1을 곱한 뒤 반환
    return -1*np.mean(roc_auc_list)

In [16]:
trials = Trials()

# fmin() 함수를 호출. max_evals 지정된 횟수만큼 반복 후 목적함수의 최솟값을 가지는 최적 입력값 추출
best = fmin(fn=bin_objective_func,
           space=xgb_search_space,
           algo=tpe.suggest,
           max_evals=50, # 최대 반복 횟수를 지정합니다
           trials=trials, rstate=np.random.default_rng(seed=109))

print('best:', best)

[0]	validation_0-auc:0.87531	validation_1-auc:0.83858                                                                  
[1]	validation_0-auc:0.89466	validation_1-auc:0.85267                                                                  
[2]	validation_0-auc:0.90672	validation_1-auc:0.86289                                                                  
[3]	validation_0-auc:0.91147	validation_1-auc:0.86775                                                                  
[4]	validation_0-auc:0.91668	validation_1-auc:0.86856                                                                  
[5]	validation_0-auc:0.91862	validation_1-auc:0.86903                                                                  
[6]	validation_0-auc:0.92302	validation_1-auc:0.87001                                                                  
[7]	validation_0-auc:0.92567	validation_1-auc:0.87243                                                                  
[8]	validation_0-auc:0.92760	validation_

[43]	validation_0-auc:0.94300	validation_1-auc:0.88186                                                                 
[44]	validation_0-auc:0.94300	validation_1-auc:0.88186                                                                 
[45]	validation_0-auc:0.94300	validation_1-auc:0.88186                                                                 
[0]	validation_0-auc:0.85050	validation_1-auc:0.83524                                                                  
[1]	validation_0-auc:0.87693	validation_1-auc:0.85517                                                                  
[2]	validation_0-auc:0.87573	validation_1-auc:0.85249                                                                  
[3]	validation_0-auc:0.88377	validation_1-auc:0.85951                                                                  
[4]	validation_0-auc:0.88615	validation_1-auc:0.86063                                                                  
[5]	validation_0-auc:0.89000	validation_

[60]	validation_0-auc:0.92210	validation_1-auc:0.87103                                                                 
[61]	validation_0-auc:0.92210	validation_1-auc:0.87103                                                                 
[62]	validation_0-auc:0.92247	validation_1-auc:0.87103                                                                 
[63]	validation_0-auc:0.92247	validation_1-auc:0.87103                                                                 
[64]	validation_0-auc:0.92287	validation_1-auc:0.87077                                                                 
[65]	validation_0-auc:0.92287	validation_1-auc:0.87077                                                                 
[66]	validation_0-auc:0.92287	validation_1-auc:0.87077                                                                 
[67]	validation_0-auc:0.92279	validation_1-auc:0.87084                                                                 
[68]	validation_0-auc:0.92295	validation

[22]	validation_0-auc:0.96350	validation_1-auc:0.86813                                                                 
[23]	validation_0-auc:0.96402	validation_1-auc:0.86807                                                                 
[24]	validation_0-auc:0.96467	validation_1-auc:0.86757                                                                 
[25]	validation_0-auc:0.96493	validation_1-auc:0.86754                                                                 
[26]	validation_0-auc:0.96590	validation_1-auc:0.86790                                                                 
[27]	validation_0-auc:0.96669	validation_1-auc:0.86731                                                                 
[28]	validation_0-auc:0.96719	validation_1-auc:0.86658                                                                 
[29]	validation_0-auc:0.96743	validation_1-auc:0.86670                                                                 
[30]	validation_0-auc:0.96841	validation

[29]	validation_0-auc:0.89460	validation_1-auc:0.87288                                                                 
[30]	validation_0-auc:0.89520	validation_1-auc:0.87281                                                                 
[31]	validation_0-auc:0.89537	validation_1-auc:0.87336                                                                 
[32]	validation_0-auc:0.89558	validation_1-auc:0.87351                                                                 
[33]	validation_0-auc:0.89598	validation_1-auc:0.87409                                                                 
[34]	validation_0-auc:0.89620	validation_1-auc:0.87477                                                                 
[35]	validation_0-auc:0.89625	validation_1-auc:0.87493                                                                 
[36]	validation_0-auc:0.89663	validation_1-auc:0.87505                                                                 
[37]	validation_0-auc:0.89700	validation

...

[47]	validation_0-auc:0.89323	validation_1-auc:0.86186                                                                 
[48]	validation_0-auc:0.89372	validation_1-auc:0.86195                                                                 
[49]	validation_0-auc:0.89362	validation_1-auc:0.86205                                                                 
[50]	validation_0-auc:0.89357	validation_1-auc:0.86190                                                                 
[51]	validation_0-auc:0.89391	validation_1-auc:0.86186                                                                 
[52]	validation_0-auc:0.89420	validation_1-auc:0.86206                                                                 
[53]	validation_0-auc:0.89437	validation_1-auc:0.86208                                                                 
[54]	validation_0-auc:0.89458	validation_1-auc:0.86213                                                                 
[55]	validation_0-auc:0.89480	validation

[83]	validation_0-auc:0.89468	validation_1-auc:0.88110                                                                 
[84]	validation_0-auc:0.89473	validation_1-auc:0.88106                                                                 
[85]	validation_0-auc:0.89469	validation_1-auc:0.88099                                                                 
[86]	validation_0-auc:0.89477	validation_1-auc:0.88109                                                                 
[87]	validation_0-auc:0.89475	validation_1-auc:0.88103                                                                 
[88]	validation_0-auc:0.89485	validation_1-auc:0.88104                                                                 
[89]	validation_0-auc:0.89499	validation_1-auc:0.88121                                                                 
[90]	validation_0-auc:0.89508	validation_1-auc:0.88125                                                                 
[91]	validation_0-auc:0.89517	validation

[27]	validation_0-auc:0.91388	validation_1-auc:0.87963                                                                 
[28]	validation_0-auc:0.91416	validation_1-auc:0.87984                                                                 
[29]	validation_0-auc:0.91416	validation_1-auc:0.87984                                                                 
[30]	validation_0-auc:0.91416	validation_1-auc:0.87984                                                                 
[31]	validation_0-auc:0.91416	validation_1-auc:0.87984                                                                 
[32]	validation_0-auc:0.91416	validation_1-auc:0.87984                                                                 
[33]	validation_0-auc:0.91416	validation_1-auc:0.87984                                                                 
[34]	validation_0-auc:0.91416	validation_1-auc:0.87984                                                                 
[35]	validation_0-auc:0.91416	validation

[20]	validation_0-auc:0.90049	validation_1-auc:0.86187                                                                 
[21]	validation_0-auc:0.90111	validation_1-auc:0.86162                                                                 
[22]	validation_0-auc:0.90143	validation_1-auc:0.86148                                                                 
[23]	validation_0-auc:0.90192	validation_1-auc:0.86149                                                                 
[24]	validation_0-auc:0.90227	validation_1-auc:0.86100                                                                 
[25]	validation_0-auc:0.90283	validation_1-auc:0.86176                                                                 
[26]	validation_0-auc:0.90303	validation_1-auc:0.86168                                                                 
[27]	validation_0-auc:0.90335	validation_1-auc:0.86199                                                                 
[28]	validation_0-auc:0.90338	validation

[56]	validation_0-auc:0.90731	validation_1-auc:0.88201                                                                 
[57]	validation_0-auc:0.90755	validation_1-auc:0.88216                                                                 
[58]	validation_0-auc:0.90774	validation_1-auc:0.88229                                                                 
[59]	validation_0-auc:0.90800	validation_1-auc:0.88241                                                                 
[60]	validation_0-auc:0.90831	validation_1-auc:0.88247                                                                 
[61]	validation_0-auc:0.90849	validation_1-auc:0.88263                                                                 
[62]	validation_0-auc:0.90884	validation_1-auc:0.88280                                                                 
[63]	validation_0-auc:0.90898	validation_1-auc:0.88292                                                                 
[64]	validation_0-auc:0.90919	validation

In [17]:
# 평가용 함수
def  get_clf_eval(y_test, pred=None, pred_proba=None):
    confusion = confusion_matrix(y_test, pred)
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)
    f1 = f1_score(y_test, pred)
#     roc_auc = roc_auc_score(y_test, pred_proba)
    
    print('오차 행렬')
    print(confusion)
 
    print('정확도: {0:.4f}, 정밀도: {1:.4f}, \
    재현율: {2:.4f}, F1: {3:.4f}'.format(accuracy, precision, recall, f1))

In [19]:
xgbo = xgb.XGBClassifier(colsample_bytree=0.6167132910325195, gamma=3.255538638467651,
                         learning_rate=0.23322945783668153,
                         max_depth=9, min_child_weight=5, random_state=109)
xgbo.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1,
              colsample_bytree=0.6167132910325195, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=3.255538638467651, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.23322945783668153, max_bin=256,
              max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
              max_depth=9, max_leaves=0, min_child_weight=5, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=0,
              num_parallel_tree=1, predictor='auto', random_state=109, ...)

In [20]:
train_pred = xgbo.predict(X_train)
train_proba = xgbo.predict_proba(X_train)

test_pred = xgbo.predict(X_test)
test_proba = xgbo.predict_proba(X_test)

val_pred = xgbo.predict(X_val)
val_proba = xgbo.predict_proba(X_val)

In [21]:
get_clf_eval(y_train, train_pred, train_proba)

오차 행렬
[[1956  438]
 [ 324 2113]]
정확도: 0.8423, 정밀도: 0.8283,     재현율: 0.8670, F1: 0.8472


In [22]:
get_clf_eval(y_test, test_pred, test_proba)

오차 행렬
[[825 242]
 [219 862]]
정확도: 0.7854, 정밀도: 0.7808,     재현율: 0.7974, F1: 0.7890


In [23]:
get_clf_eval(y_val, val_pred, val_proba)

오차 행렬
[[620 176]
 [140 675]]
정확도: 0.8038, 정밀도: 0.7932,     재현율: 0.8282, F1: 0.8103


In [26]:
fi = pd.DataFrame(xgbo.feature_importances_)

In [27]:
fi.to_csv('fi.csv')